In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
url = 'https://docs.google.com/spreadsheets/d/1_y0_LJmSY6sNx8qd51T70n0oa_ugN50AVFKuJmXO1-s/export?format=csv&gid=2042605142'
approval = pd.read_csv(url)
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,90663,1528,AtlasIntel,NaN,NaN,NaN,NaN,NaN,AtlasIntel,Online Ad,...,NaN,https://cdn.atlasintel.org/e734ffb6-e175-4467-...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,51.8,43.5
131,90675,1805,Verasight,2265,Strength In Numbers,NaN,NaN,NaN,Verasight,Text/Online Ad/Email,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,47.0,42.0
132,90674,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,https://d3nkl3psvxxpe9.cloudfront.net/document...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,39.0,35.0
133,90674,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,https://d3nkl3psvxxpe9.cloudfront.net/document...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,42.0


In [7]:
approval = approval[~approval['poll_id'].isna()]
approval

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,90663,1528,AtlasIntel,NaN,NaN,NaN,NaN,NaN,AtlasIntel,Online Ad,...,NaN,https://cdn.atlasintel.org/e734ffb6-e175-4467-...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,51.8,43.5
131,90675,1805,Verasight,2265,Strength In Numbers,NaN,NaN,NaN,Verasight,Text/Online Ad/Email,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,47.0,42.0
132,90674,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,https://d3nkl3psvxxpe9.cloudfront.net/document...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,39.0,35.0
133,90674,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,https://d3nkl3psvxxpe9.cloudfront.net/document...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,42.0


In [9]:
approval.columns.values

array(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'sponsor_candidate_id', 'sponsor_candidate',
       'sponsor_candidate_party', 'display_name', 'mode', 'state',
       'start_date', 'end_date', 'internal', 'partisan', 'question_id',
       'sample_size', 'population', 'subpopulation', 'population_full',
       'tracking', 'created_at', 'updated_at', 'notes', 'url',
       'url_article', 'url_topline', 'url_crosstab', 'race_id', 'cycle',
       'office_type', 'seat_name', 'election_date', 'stage', 'dem', 'rep'],
      dtype=object)

In [11]:
# morris_to_silver_dict = {
#     'HarrisX/Harris Poll':'Harris Insights & Analytics',
#     'Clarity': 'Clarity Campaign Labs',
#     'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
#     'Monmouth': 'Monmouth University',
#     'HarrisX': 'Harris Insights & Analytics',
#     'UMass Amherst/YouGov': 'YouGov',
#     'Emerson': 'Emerson College',
#     'Marquette Law School': 'Marquette University Law School',
#     'NewsNation/Decision Desk HQ': 'NewsNation',
#     'CNN/SSRS': 'SSRS',
#     'Suffolk': 'Suffolk University',
#     'Harris Poll': 'Harris Insights & Analytics',
#     'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
#     'Winston': 'The Winston Group',
#     'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
#     'Quinnipiac': 'Quinnipiac University',
#     'Siena/NYT': 'The New York Times/Siena College',
#     'Noble Predictive Insights': 'OH Predictive Insights',
#     'McLaughlin': 'McLaughlin & Associates'
# }

In [13]:
pollster_key = pd.DataFrame(approval[['pollster', 'display_name']].value_counts()).reset_index()
pollster_key.head()

,pollster,display_name,count
0,YouGov,YouGov,33
1,McLaughlin,McLaughlin & Associates,18
2,Cygnal,Cygnal,11
3,Echelon Insights,Echelon Insights,10
4,Quantus Insights,Quantus Insights,9


In [15]:
pollster_key

,pollster,display_name,count
0,YouGov,YouGov,33
1,McLaughlin,McLaughlin & Associates,18
2,Cygnal,Cygnal,11
3,Echelon Insights,Echelon Insights,10
4,Quantus Insights,Quantus Insights,9
5,RMG Research,RMG Research,8
6,Clarity,Clarity Campaign Labs,6
7,Verasight,Verasight,5
8,Emerson,Emerson College,4
9,AtlasIntel,AtlasIntel,3


In [17]:
# pollsters_short = np.unique(approval['pollster'])
# pollsters_long = np.unique(approval['display_name'])

In [19]:
# pollsters_short, pollsters_long

In [21]:
# pollsters_short.shape, pollsters_long.shape

In [23]:
morris_to_silver_dict = {short: long for short, long in zip(approval['pollster'], approval['display_name'])}
morris_to_silver_dict

{'Cygnal': 'Cygnal',
 'McLaughlin': 'McLaughlin & Associates',
 'RMG Research': 'RMG Research',
 'Quantus Insights': 'Quantus Insights',
 'Fabrizio/Impact': 'Fabrizio, Lee & Associates/Impact Research',
 'Clarity': 'Clarity Campaign Labs',
 'Echelon Insights': 'Echelon Insights',
 'co/efficient': 'co/efficient',
 'YouGov': 'YouGov',
 'Public Opinion Strategies': 'Public Opinion Strategies',
 'Emerson': 'Emerson College',
 'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
 'Normington, Petts & Associates': 'Normington, Petts & Associates',
 'Yale Youth Poll': 'Yale University Institution for Social and Policy Studies',
 'Winston': 'The Winston Group',
 'RealClear Opinion Research': 'RealClear Opinion Research',
 'Noble Predictive Insights': 'Noble Predictive Insights',
 'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
 'Siena/NYT': 'The New York Times/Siena College',
 'NewsNation/Decision Desk HQ': 'NewsNation',
 'Big Data Poll': 'Big Data Poll',
 'Verasight': 'Vera

In [77]:
morris_to_silver_dict.update({
    'HarrisX/Harris Poll':'Harris Insights & Analytics',
    'Clarity': 'Clarity Campaign Labs',
    'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
    'Monmouth': 'Monmouth University',
    'HarrisX': 'Harris Insights & Analytics',
    'UMass Amherst/YouGov': 'YouGov',
    'Emerson': 'Emerson College',
    'Marquette Law School': 'Marquette University Law School',
    'NewsNation/Decision Desk HQ': 'NewsNation',
    'CNN/SSRS': 'SSRS',
    'Suffolk': 'Suffolk University',
    'Harris Poll': 'Harris Insights & Analytics',
    'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
    'Winston': 'Verasight',
    'Verasight': 'Verasight',
    'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
    'Quinnipiac': 'Quinnipiac University',
    'Siena/NYT': 'The New York Times/Siena College',
    'Noble Predictive Insights': 'OH Predictive Insights',
    'McLaughlin': 'McLaughlin & Associates',
    'Verasight/The Argument': 'Verasight'
})

In [78]:
morris_to_silver_dict

{'Cygnal': 'Cygnal',
 'McLaughlin': 'McLaughlin & Associates',
 'RMG Research': 'RMG Research',
 'Quantus Insights': 'Quantus Insights',
 'Fabrizio/Impact': 'Fabrizio Lee & Associates/Impact Research',
 'Clarity': 'Clarity Campaign Labs',
 'Echelon Insights': 'Echelon Insights',
 'co/efficient': 'co/efficient',
 'YouGov': 'YouGov',
 'Public Opinion Strategies': 'Public Opinion Strategies',
 'Emerson': 'Emerson College',
 'Hart/POS': 'Hart Research Associates/Public Opinion Strategies',
 'Normington, Petts & Associates': 'Normington, Petts & Associates',
 'Yale Youth Poll': 'Yale University Institution for Social and Policy Studies',
 'Winston': 'Verasight',
 'RealClear Opinion Research': 'RealClear Opinion Research',
 'Noble Predictive Insights': 'OH Predictive Insights',
 'Beacon/Shaw': 'Beacon Research/Shaw & Co. Research',
 'Siena/NYT': 'The New York Times/Siena College',
 'NewsNation/Decision Desk HQ': 'NewsNation',
 'Big Data Poll': 'Big Data Poll',
 'Verasight': 'Verasight',
 'Fa

In [79]:
approval['pollster'] = approval['pollster'].replace(morris_to_silver_dict)
approval = approval.rename({'population':'population_empty'}, axis='columns')
approval = approval.rename({'population_full':'population'}, axis='columns')
approval.head(7)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,url_topline,url_crosstab,race_id,cycle,office_type,seat_name,election_date,stage,dem,rep
0,89437,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2024/12...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,48.4
1,89445,294,McLaughlin & Associates,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,42.0,47.0
2,89491,1347,Cygnal,NaN,NaN,NaN,NaN,NaN,Cygnal,NaN,...,NaN,https://www.cygn.al/wp-content/uploads/2025/01...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.6,46.8
3,89497,1554,RMG Research,2178,Napolitan Institute,NaN,NaN,NaN,RMG Research,Online Opt-In Panel,...,NaN,NaN,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,44.0,51.0
4,89511,1895,Quantus Insights,2184,TrendingPolitics,NaN,NaN,NaN,Quantus Insights,Online Opt-In Panel,...,NaN,https://drive.google.com/file/d/1d1e9IDGzoiE2S...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,45.0,48.0
5,89530,294,McLaughlin & Associates,NaN,NaN,NaN,NaN,NaN,McLaughlin & Associates,Online Opt-In Panel,...,NaN,https://mclaughlinonline.com/pols/wp-content/u...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,40.0,48.0
6,89555,1699,Fabrizio Lee & Associates/Impact Research,42,AARP,NaN,NaN,NaN,"Fabrizio, Lee & Associates/Impact Research",Online Opt-In Panel/Text-to-Web,...,NaN,https://www.aarp.org/content/dam/aarp/research...,16444,2026,U.S. House,Generic Ballot,2026-11-03,general,43.0,43.0


In [80]:
approval.shape

(129, 36)

In [81]:
approval = approval[approval['partisan'].isna()]# [pd.to_datetime(approval['end_date']) >= pd.to_datetime('2023-12-01')]
approval.shape

(107, 36)

In [82]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [83]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [84]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(105, 36)

In [85]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval.shape

(105, 38)

In [86]:
approval = approval[(~approval['sample_size'].isna())]
approval.shape

(104, 38)

In [87]:
approval['pollster'].value_counts()

pollster
YouGov                                                       31
Cygnal                                                       11
Echelon Insights                                              9
RMG Research                                                  8
McLaughlin & Associates                                       8
Verasight                                                     5
Clarity Campaign Labs                                         5
Emerson College                                               4
Fabrizio Lee & Associates/Impact Research                     3
Quantus Insights                                              3
Hart Research Associates/Public Opinion Strategies            2
AtlasIntel                                                    2
Ipsos                                                         1
American Pulse Research & Polling                             1
J.L. Partners                                                 1
Fabrizio Ward                  

In [88]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [89]:
# house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+\d.\d\d.*@@([\d\.\+\-]+)')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,-1.999072
1,D,1.017374
2,D,0.632896
3,D,0.791334
4,R,-0.597041
...,...,...
93,R,-2.443137
94,R,-0.446172
95,R,-3.234717
96,R,-4.939213


In [90]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[1]
house_eff

,0,1,house_effect
0,-1,-1.999072,-1.999072
1,1,1.017374,1.017374
2,1,0.632896,0.632896
3,1,0.791334,0.791334
4,-1,-0.597041,-0.597041
...,...,...,...
93,-1,-2.443137,-2.443137
94,-1,-0.446172,-0.446172
95,-1,-3.234717,-3.234717
96,-1,-4.939213,-4.939213


In [91]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-1.999072
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.017374
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.632896
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.791334
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.597041
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.443137
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.446172
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.234717


In [92]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['dem_adj'] = df['dem'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['rep_adj'] = df['rep'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['dem_adj'] = df['dem']
        df['rep_adj'] = df['rep']
    return df

In [93]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,Predictive Plus-Minus,dem_adj,rep_adj
9,89579,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587
10,89579,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.750413,41.249587
23,89690,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.750413,36.249587
24,89690,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
38,89897,568,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,89719,1800,OH Predictive Insights,2013,The Center Square,NaN,NaN,NaN,Noble Predictive Insights,Online Opt-In Panel/Text-to-Web,...,U.S. House,Generic Ballot,2026-11-03,general,45.0,42.0,OH Predictive Insights,-0.225885,43.935200,43.064800
22,89744,1903,Yale University Institution for Social and Pol...,NaN,NaN,NaN,NaN,NaN,Yale University Institution for Social and Pol...,Online Opt-In Panel,...,U.S. House,Generic Ballot,2026-11-03,general,43.3,41.7,NaN,NaN,43.300000,41.700000
19,89821,1107,"Normington, Petts & Associates",543,Third Way,NaN,NaN,NaN,"Normington, Petts & Associates",NaN,...,U.S. House,Generic Ballot,2026-11-03,general,50.0,50.0,"Normington, Petts & Associates",0.837728,50.000000,50.000000
13,89658,379,Public Opinion Strategies,2255,Paragon Health Institute,NaN,NaN,NaN,Public Opinion Strategies,Live Phone,...,U.S. House,Generic Ballot,2026-11-03,general,44.0,46.0,Public Opinion Strategies,-0.246974,46.071409,43.928591


In [94]:
approval = approval_adj.copy()
approval.shape

(104, 40)

In [95]:
approval['pollster_id'] = pd.factorize(approval['pollster'] + approval['sponsors'].fillna('none'))[0]
approval.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,Predictive Plus-Minus,dem_adj,rep_adj
9,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587
10,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.750413,41.249587
23,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.750413,36.249587
24,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
38,89897,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587


In [96]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id')]
    ndup = data[~data.duplicated(subset='poll_id')]
    #try:
    dup_a = dup[dup['population'].isin(['lv'])]
#     except ValueError:
#         print(dup['population'].isin(['a']))
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id')]
    ndup = df[~df.duplicated(subset='poll_id')]
    dup_lv = dup[dup['population'].isin(['rv'])]
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [97]:
approval = approval.rename({'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [98]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [99]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,office_type,seat_name,election_date,stage,dem,rep,Pollster,pred_plus_minus,dem_adj,rep_adj
9,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587
10,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,42.0,41.0,YouGov,0.047635,41.750413,41.249587
23,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,39.0,36.0,YouGov,0.047635,38.750413,36.249587
24,89690,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587
38,89897,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,U.S. House,Generic Ballot,2026-11-03,general,43.0,42.0,YouGov,0.047635,42.750413,42.249587


In [100]:
approval.shape

(104, 40)

In [101]:
# net: negative --> dem advantage, positive --> rep advantage
approval['net'] = approval['rep'] - approval['dem']
approval.head(1)

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,display_name,mode,...,seat_name,election_date,stage,dem,rep,Pollster,pred_plus_minus,dem_adj,rep_adj,net
9,89579,0,YouGov,352,The Economist,NaN,NaN,NaN,YouGov,Online Matched Sample,...,Generic Ballot,2026-11-03,general,35.0,37.0,YouGov,0.047635,34.750413,37.249587,2.0


In [102]:
approval['url'] = approval['url_crosstab'].fillna(approval['url_topline']).fillna(approval['url_article'].fillna(''))

In [143]:
approval_polls = approval[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'dem_adj', 'rep_adj', 'net', 'pred_plus_minus', 'poll_id', 'url', 'sponsors', 'pollster_id']]
approval_polls

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors,pollster_id
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.750413,37.249587,2.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
10,YouGov,2025-02-16,2025-02-18,1451.0,rv,42.0,41.0,41.750413,41.249587,-1.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.750413,36.249587,-3.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
24,YouGov,2025-04-05,2025-04-08,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
38,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89897,https://today.yougov.com/topics/politics/track...,The Economist,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,OH Predictive Insights,2025-04-15,2025-04-18,2527.0,rv,45.0,42.0,43.935200,43.064800,-3.0,-0.225885,89719,https://bloximages.newyork1.vip.townnews.com/t...,The Center Square,25
22,Yale University Institution for Social and Pol...,2025-04-01,2025-04-03,4100.0,rv,43.3,41.7,43.300000,41.700000,-1.6,NaN,89744,https://docs.google.com/spreadsheets/d/1JhSWPV...,NaN,26
19,"Normington, Petts & Associates",2025-03-12,2025-03-19,800.0,lv,50.0,50.0,50.000000,50.000000,0.0,0.837728,89821,https://thirdway.imgix.net/downloads/democrats...,Third Way,27
13,Public Opinion Strategies,2025-02-25,2025-03-02,1000.0,rv,44.0,46.0,46.071409,43.928591,2.0,-0.246974,89658,https://paragoninstitute.org/medicaid/paragon-...,Paragon Health Institute,28


In [145]:
def zone_flood_checker(data, pid, date):
    """
    Check whether there are multiple polls from the same pollster + same sponsor (if applicable) within the last two weeks, 
    which could indicate "flooding the zone." Return number of polls from same pollster/sponsor in last two weeks.

    :param data: Polls dataframe.
    :param pid: Pollster ID, specified in the polls dataframe as the column pollster_id.
    :param date: Date at which specified poll is conducted. Corresponds to column end_date in polls dataframe.
    """
    thres = date - pd.Timedelta(weeks=2)
    df = data[ (data['pollster_id'] == pid) & (pd.to_datetime(data['end_date']) >= thres) ]
    return max(df.shape[0], 1)

In [147]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    # state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    # linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    linear_weights = delta.map(lambda x: (1 - x/60) if x < 60 else 0)
    exp_weights = 0.1**(delta/(45))
    state_pivot['time_weights'] =  0.1 * linear_weights + 0.9 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual):
            return 0.2
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'rv':
            return 0.8
        elif population == 'a':
            return 0.5
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Multiple polls in short window (aka "flooding the zone")
    def zone_flood_weight(pid, date):
        num_polls = zone_flood_checker(approval_polls, pid, date)
        return 1 / np.sqrt(num_polls)
    state_pivot['zone_flood_weight'] = state_pivot[['pollster_id', 'end_date']].apply(
        lambda x: zone_flood_weight(x['pollster_id'], x['end_date']),
        axis=1
    )

    # Gather the weights together
    state_pivot['total_weights'] = (state_pivot['sample_size_weights'] * state_pivot['time_weights'] * 
                                    state_pivot['quality_weights'] * state_pivot['population_weights'] * state_pivot['zone_flood_weight'])
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [149]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,...,poll_id,url,sponsors,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
67,Echelon Insights,2025-07-10,2025-07-14,1084.0,lv,47.0,48.0,47.000000,48.000000,1.0,...,90309,https://echelonin.wpenginepowered.com/wp-conte...,NaN,3,0.009729,0.034042,1.181365,1.0,0.577350,5.460007e-03
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.750413,37.249587,2.0,...,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0,0.011831,0.000019,0.964021,0.5,0.185695,4.962892e-07
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.750413,36.249587,-3.0,...,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0,0.012330,0.000238,0.964021,0.5,0.192450,6.577691e-06
38,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,...,89897,https://today.yougov.com/topics/politics/track...,The Economist,0,0.011664,0.001939,0.964021,0.8,0.200000,8.431421e-05
42,YouGov,2025-05-23,2025-05-26,1486.0,rv,43.0,41.0,42.750413,41.249587,-2.0,...,90016,https://today.yougov.com/topics/politics/track...,The Economist,0,0.011391,0.002774,0.964021,0.8,0.200000,1.178100e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,OH Predictive Insights,2025-04-15,2025-04-18,2527.0,rv,45.0,42.0,43.935200,43.064800,-3.0,...,89719,https://bloximages.newyork1.vip.townnews.com/t...,The Center Square,25,0.014854,0.000397,1.189767,0.8,1.000000,1.356364e-04
22,Yale University Institution for Social and Pol...,2025-04-01,2025-04-03,4100.0,rv,43.3,41.7,43.300000,41.700000,-1.6,...,89744,https://docs.google.com/spreadsheets/d/1JhSWPV...,NaN,26,0.016185,0.000184,0.100000,0.8,1.000000,5.765527e-06
19,"Normington, Petts & Associates",2025-03-12,2025-03-19,800.0,lv,50.0,50.0,50.000000,50.000000,0.0,...,89821,https://thirdway.imgix.net/downloads/democrats...,Third Way,27,0.008358,0.000086,0.200000,1.0,1.000000,3.454857e-06
13,Public Opinion Strategies,2025-02-25,2025-03-02,1000.0,rv,44.0,46.0,46.071409,43.928591,2.0,...,89658,https://paragoninstitute.org/medicaid/paragon-...,Paragon Health Institute,28,0.009344,0.000036,1.209225,0.8,1.000000,7.828426e-06


In [150]:
approval_polls['end_date'].min()

'2024-12-11'

In [151]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 20), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['rep_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['dem_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['rep_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['dem_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Republicans':yes_averages, 'Democrats':no_averages}), pd.DataFrame({'Date':date_range, 'Republicans':yes_stdevs, 'Democrats':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [152]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Republicans,Democrats
0,2025-01-20,47.787791,44.654648
1,2025-01-21,47.787507,44.654576
2,2025-01-22,47.787230,44.654483
3,2025-01-23,47.786960,44.654368
4,2025-01-24,47.786698,44.654227
...,...,...,...
235,2025-09-12,42.850126,45.538216
236,2025-09-13,42.850355,45.534973
237,2025-09-14,42.850581,45.531386
238,2025-09-15,42.343551,45.062270


In [153]:
polls_df.shape

(94, 21)

In [154]:
display_df = polls_df[['pollster', 'start_date', 'end_date', 'sample_size', 'population', 'dem', 'rep', 'net',
                      'total_weights', 'url', 'sponsors']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 'total_weights':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population'].map(lambda x: x.upper()).astype(str)
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-09-12,2025-09-15,1567 A,5.304488e-02,37.0,33.0,-4.0
1,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-09-05,2025-09-08,1644 A,3.377301e-02,39.0,36.0,-3.0
2,"<a href=""https://www.cygn.al/wp-content/upload...",2025-09-02,2025-09-03,1500 LV,2.659600e-01,47.6,45.2,-2.4
3,"<a href=""https://www.scribd.com/document/91189...",2025-08-29,2025-09-02,1136 RV,9.585995e-02,44.0,40.0,-4.0
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-08-29,2025-09-02,1691 A,2.584371e-02,38.0,34.0,-4.0
...,...,...,...,...,...,...,...,...
89,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,1.073996e-07,40.0,48.0,8.0
90,"<a href=""https://napolitannews.org/posts/gener...",2025-01-15,2025-01-16,1000 RV,1.848172e-07,44.0,51.0,7.0
91,"<a href=""https://www.cygn.al/wp-content/upload...",2025-01-09,2025-01-12,1500 LV,4.538368e-07,44.6,46.8,2.2
92,"<a href=""https://mclaughlinonline.com/pols/wp-...",2024-12-11,2024-12-16,1000 LV,1.171314e-08,42.0,47.0,5.0


In [155]:
display_df.to_csv('generic_ballot_polls_display_table.csv')

In [156]:
approval_avg['Net'] = approval_avg['Republicans'] - approval_avg['Democrats']
approval_avg

,Date,Republicans,Democrats,Net
0,2025-01-20,47.787791,44.654648,3.133143
1,2025-01-21,47.787507,44.654576,3.132931
2,2025-01-22,47.787230,44.654483,3.132747
3,2025-01-23,47.786960,44.654368,3.132592
4,2025-01-24,47.786698,44.654227,3.132471
...,...,...,...,...
235,2025-09-12,42.850126,45.538216,-2.688089
236,2025-09-13,42.850355,45.534973,-2.684618
237,2025-09-14,42.850581,45.531386,-2.680805
238,2025-09-15,42.343551,45.062270,-2.718719


In [157]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Republicans_x'] + 1.96*approval_trendline['Republicans_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Republicans_x'] - 1.96*approval_trendline['Republicans_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Democrats_x'] + 1.96*approval_trendline['Democrats_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Democrats_x'] - 1.96*approval_trendline['Democrats_y']
approval_trendline = approval_trendline.rename({
    'Republicans_x':'approve_mean',
    'Democrats_x':'disapprove_mean',
}, axis='columns').drop(['Republicans_y', 'Democrats_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-20,47.787791,44.654648,51.629130,43.946452,45.467443,43.841854
1,2025-01-21,47.787507,44.654576,51.629477,43.945538,45.467464,43.841689
2,2025-01-22,47.787230,44.654483,51.629972,43.944488,45.467490,43.841477
3,2025-01-23,47.786960,44.654368,51.630628,43.943292,45.467523,43.841212
4,2025-01-24,47.786698,44.654227,51.631458,43.941938,45.467563,43.840891
...,...,...,...,...,...,...,...
235,2025-09-12,42.850126,45.538216,49.905931,35.794321,51.897039,39.179392
236,2025-09-13,42.850355,45.534973,49.907772,35.792937,51.890967,39.178979
237,2025-09-14,42.850581,45.531386,49.909769,35.791394,51.884292,39.178481
238,2025-09-15,42.343551,45.062270,50.406597,34.280505,52.343644,37.780896


In [158]:
# remember: approve = Republicans, disapprove = Democrats - kept for DataWrapper setup simplicity
# Export generic ballot averages
approval_trendline.to_csv('generic_ballot_stats.csv')

In [159]:
# line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# # scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
# line.update_traces(hovertemplate=None)
# line.update_layout(hovermode='x unified')
# # scatter.update_traces(hovertemplate=None, hoverinfo='skip')
# fig_approve_CI = go.Figure([
#     go.Scatter(
#         name='Approve CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Approve CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
#         mode='lines',
#         marker=dict(color='#8be8f2'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])
# fig_disapprove_CI = go.Figure([
#     go.Scatter(
#         name='Disapprove CI Upper Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),
#     go.Scatter(
#         name='Disapprove CI Lower Bound',
#         x = approval_avg['Date'],
#         y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
#         mode='lines',
#         marker=dict(color='#a3cbec'),
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     ),  
# ])

# fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
# fig.update_layout(
#     title='Congressional Approval Rating',
#     xaxis_title='Date',
#     yaxis_title='%',
#     legend_title='Legend',
#     hovermode='x unified'
# )
# fig.show()

In [160]:
approval_polls.head()

,pollster,start_date,end_date,sample_size,population,dem,rep,dem_adj,rep_adj,net,pred_plus_minus,poll_id,url,sponsors,pollster_id
9,YouGov,2025-02-16,2025-02-18,1603.0,a,35.0,37.0,34.750413,37.249587,2.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
10,YouGov,2025-02-16,2025-02-18,1451.0,rv,42.0,41.0,41.750413,41.249587,-1.0,0.047635,89579,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
23,YouGov,2025-04-05,2025-04-08,1741.0,a,39.0,36.0,38.750413,36.249587,-3.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
24,YouGov,2025-04-05,2025-04-08,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89690,https://d3nkl3psvxxpe9.cloudfront.net/document...,The Economist,0
38,YouGov,2025-05-16,2025-05-19,1558.0,rv,43.0,42.0,42.750413,42.249587,-1.0,0.047635,89897,https://today.yougov.com/topics/politics/track...,The Economist,0


In [161]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['rep_adj'] - data['dem_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [162]:
net_approval = net_col(approval_polls, datetime.date(2025, 1, 20))
net_approval

,Date,net,net_std
0,2025-01-20,3.133143,2.348178
1,2025-01-21,3.132931,2.348610
2,2025-01-22,3.132747,2.349141
3,2025-01-23,3.132592,2.349778
4,2025-01-24,3.132471,2.350531
...,...,...,...
235,2025-09-12,-2.688089,1.952449
236,2025-09-13,-2.684618,1.942206
237,2025-09-14,-2.680805,1.930880
238,2025-09-15,-2.718719,1.878610


In [163]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-20,3.133143,2.348178,7.735572,-1.469286
1,2025-01-21,3.132931,2.348610,7.736207,-1.470345
2,2025-01-22,3.132747,2.349141,7.737062,-1.471569
3,2025-01-23,3.132592,2.349778,7.738157,-1.472973
4,2025-01-24,3.132471,2.350531,7.739512,-1.474570
...,...,...,...,...,...
235,2025-09-12,-2.688089,1.952449,1.138712,-6.514890
236,2025-09-13,-2.684618,1.942206,1.122105,-6.491341
237,2025-09-14,-2.680805,1.930880,1.103720,-6.465331
238,2025-09-15,-2.718719,1.878610,0.963357,-6.400795


In [164]:
net_approval.to_csv('net_generic_ballot.csv')